In [10]:
import spotipy
import spotipy.util as util


client_id = 'b69a9985fa8842deb0691b2d0e3f0b69'
client_secret = 'd9e9ae2924174c139a5a9ccb303f9f3a'
redirect_uri = 'http://localhost/'

username = '22mrmbu7oumkrb56tcsclawdi'
scope = 'user-library-read'

token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
    tracks = []
    offset = 0
    while True:
        track_set = sp.current_user_saved_tracks(limit=50, offset=offset)['items']
        if track_set:
            tracks += track_set
        else:
            break
        offset += 50
    ids = []
    
    for i in range(len(tracks)):
        tracks[i] = tracks[i]['track']
        ids.append(tracks[i]['id'])


In [13]:
import pandas as pd

data = {
    'acousticness': [],
    'danceability': [],
#     'duration_ms': [],
    'energy': [],
    'instrumentalness': [],
#     'key': [],
#     'liveness': [],
    'loudness': [],
#     'mode': [],
    'speechiness': [],
    'tempo': [],
#     'time_signature': [],
    'valence': [],
}

def chunks(n, x):
    return [x[i:i + n] for i in range(0, len(x), n)]

for chunk in chunks(100, ids):
    track_set = sp.audio_features(tracks=chunk)
    for track in track_set:
        for key in data:
            data[key].append(track[key])

artist_genres = dict()
track_artists = []
for track in tracks:
    artists = [x['id'] for x in track['artists']]
    track_artists.append(artists)
    for artist in artists:
        if artist not in artist_genres:
            artist_genres[artist] = None

for chunk in chunks(50, list(artist_genres.keys())):
    for artist in sp.artists(chunk)['artists']:
        id = artist['id']
        artist_genres[id] = artist['genres']

data['genres'] = []
for artists in track_artists:
    genres = set()
    for id in artists:
        genres.update(artist_genres[id])
    data['genres'].append(genres)
    
df = pd.DataFrame(data=data, index=ids)

In [14]:
display(df)

,acousticness,danceability,energy,instrumentalness,loudness,speechiness,tempo,valence,genres
7MCgEv2ovQuxfijz2WZ4kH,0.033000,0.6150,0.826,0.000070,-3.774,0.1450,121.940,0.4890,"{tropical house, dutch rock, deep tropical hou..."
6OASuyRPP9Z7Ng6v3fCby7,0.008490,0.4640,0.966,0.045500,-2.118,0.2470,149.719,0.4380,"{electronic trap, vapor twitch, brostep}"
2rfHCPzXCEYL5B7Z2j4yiE,0.023400,0.6360,0.933,0.013100,-3.349,0.5150,170.276,0.3920,"{electronic trap, vapor twitch, brostep}"
5EX8gks8V2wDZanRGAy8pm,0.004670,0.6650,0.951,0.000072,-2.204,0.3390,75.006,0.4820,"{electronic trap, vapor twitch, brostep}"
5eJQn2KuywbQxYBwZA4qRq,0.000717,0.7060,0.913,0.004580,-3.236,0.2130,127.927,0.4780,"{electronic trap, vapor twitch, brostep}"
1AjfhyWbDYvrja6Z2SScxg,0.006990,0.5920,0.964,0.382000,-3.254,0.3570,74.896,0.4790,"{electronic trap, vapor twitch, brostep}"
1AIHqJAaV8If2DQ29BotoL,0.034600,0.4970,0.913,0.014000,-3.385,0.2890,115.559,0.1820,"{electronic trap, vapor twitch, brostep}"
76ovB1o23o87WCV2wQqOfq,0.023000,0.5010,0.898,0.008410,-4.911,0.3160,106.393,0.2290,"{electronic trap, vapor twitch, brostep}"
08aV7tJySewyN66bLeaJZl,0.028600,0.5110,0.733,0.004910,-2.802,0.0455,80.000,0.1360,"{electronic trap, vapor twitch, brostep}"
4FtjMbx5rC3fUqcBJXhu1G,0.000429,0.6600,0.798,0.812000,-5.549,0.0394,123.013,0.4200,"{electronic, house, deep house, tech house, fl..."


In [15]:
import numpy as np
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

features = df.loc[:, df.columns != 'genres']
display(features)
g = sns.PairGrid(features)
g.map_diag(sns.kdeplot)
g.map_offdiag(sns.kdeplot, n_levels=10);

X = preprocessing.scale(features.values)
for x in X.transpose():
    sns.distplot(x)
    plt.show()

,acousticness,danceability,energy,instrumentalness,loudness,speechiness,tempo,valence
7MCgEv2ovQuxfijz2WZ4kH,0.033000,0.6150,0.826,0.000070,-3.774,0.1450,121.940,0.4890
6OASuyRPP9Z7Ng6v3fCby7,0.008490,0.4640,0.966,0.045500,-2.118,0.2470,149.719,0.4380
2rfHCPzXCEYL5B7Z2j4yiE,0.023400,0.6360,0.933,0.013100,-3.349,0.5150,170.276,0.3920
5EX8gks8V2wDZanRGAy8pm,0.004670,0.6650,0.951,0.000072,-2.204,0.3390,75.006,0.4820
5eJQn2KuywbQxYBwZA4qRq,0.000717,0.7060,0.913,0.004580,-3.236,0.2130,127.927,0.4780
1AjfhyWbDYvrja6Z2SScxg,0.006990,0.5920,0.964,0.382000,-3.254,0.3570,74.896,0.4790
1AIHqJAaV8If2DQ29BotoL,0.034600,0.4970,0.913,0.014000,-3.385,0.2890,115.559,0.1820
76ovB1o23o87WCV2wQqOfq,0.023000,0.5010,0.898,0.008410,-4.911,0.3160,106.393,0.2290
08aV7tJySewyN66bLeaJZl,0.028600,0.5110,0.733,0.004910,-2.802,0.0455,80.000,0.1360
4FtjMbx5rC3fUqcBJXhu1G,0.000429,0.6600,0.798,0.812000,-5.549,0.0394,123.013,0.4200


<Figure size 2000x2000 with 72 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [16]:
import sklearn.cluster

clusters = 5
df['cluster'] = sklearn.cluster.SpectralClustering(n_clusters=5).fit_predict(X)

In [20]:
clusters = dict()
for cluster in df['cluster'].unique():
    df_cluster = df[ df['cluster']==cluster ]
    genre_count = dict()
    for genres in df_cluster['genres']:
        for genre in genres:
            if genre in genre_count:
                genre_count[genre] += 1
            else:
                genre_count[genre] = 1
    name = ','.join(sorted(genre_count, key=genre_count.get, reverse=True)[:3])
    clusters[name] = df_cluster.index.values
print(clusters.keys())

dict_keys(['electro house,edm,big room', 'electro house,edm,electronic trap', 'edm,electro house,brostep', 'edm,electro house,progressive house', 'tech house,edm,progressive house'])


In [17]:
classified = [[] for i in range(clusters)]
ids = df.index.values
for i in range(len(df)):
    cluster = model.predict(X[0].reshape(1, -1))[0]
    classified[cluster].append(ids[i])
for i in classified:
    print(len(i))

AttributeError: 'SpectralClustering' object has no attribute 'predict'

In [9]:
import json
print(json.dumps(tracks, sort_keys=True, indent=4))

[
    {
        "acousticness": 0.0119,
        "analysis_url": "https://api.spotify.com/v1/audio-analysis/0YdZ8DaTnj1sF2g2KMlS36",
        "danceability": 0.405,
        "duration_ms": 256629,
        "energy": 0.39,
        "id": "0YdZ8DaTnj1sF2g2KMlS36",
        "instrumentalness": 0.925,
        "key": 1,
        "liveness": 0.143,
        "loudness": -11.385,
        "mode": 0,
        "speechiness": 0.0302,
        "tempo": 77.495,
        "time_signature": 4,
        "track_href": "https://api.spotify.com/v1/tracks/0YdZ8DaTnj1sF2g2KMlS36",
        "type": "audio_features",
        "uri": "spotify:track:0YdZ8DaTnj1sF2g2KMlS36",
        "valence": 0.0358
    },
    {
        "acousticness": 0.00894,
        "analysis_url": "https://api.spotify.com/v1/audio-analysis/2OgAlPzPvMTATMrKlNGT6F",
        "danceability": 0.489,
        "duration_ms": 246133,
        "energy": 0.767,
        "id": "2OgAlPzPvMTATMrKlNGT6F",
        "instrumentalness": 0.0012,
        "key": 8,
        "live